In [371]:
from linear_algebra import squared_distance, Vector
from typing import List

In [372]:
inputs: List[List[float]] = [[-14,-5],[13,13],[20,23],[-19,-11],[-9,-16],[21,27],[-49,15],[26,13],[-46,5],[-34,-1],[11,15],
                             [-49,0],[-22,-16],[19,28],[-12,-8],[-13,-19],[-41,8],[-11,-6],[-25,-9],[-18,-3]]   
inputs

[[-14, -5],
 [13, 13],
 [20, 23],
 [-19, -11],
 [-9, -16],
 [21, 27],
 [-49, 15],
 [26, 13],
 [-46, 5],
 [-34, -1],
 [11, 15],
 [-49, 0],
 [-22, -16],
 [19, 28],
 [-12, -8],
 [-13, -19],
 [-41, 8],
 [-11, -6],
 [-25, -9],
 [-18, -3]]

In [373]:
assert squared_distance([-14, 5], [-14, 5]) == 0

In [374]:
import random
from typing import Dict
from collections import namedtuple

def random_centerpoints(inputs: List[Vector], k: int) -> List[Vector]:
    """Randomly chooses k centerpoints""" 
    return [random.choice(inputs) for _ in range(k)]

def compare_membership(v1: Vector, v2: Vector) -> bool:
    assert len(v1) == len(v2)
    return all(x1 == x2 for x1, x2 in zip(v1, v2))

def random_assignments(inputs: List[Vector], k: int) -> List[int]:
    """ assigns a random member ship to all inputs
    """
    return [random.randrange(3) for _ in inputs]

def assigments_check(assignments: Dict[int,List[Vector]], k: int, inputs: List[Vector]) -> Dict[int,List[Vector]]:
    """ Checks if assigment dict index:[Vectors] have the correct number of k keys
        if not assigns a random input value
    """
    for k in range(k):
        if not k in assignments:
            assignments[k] = [random.choice(inputs)]
    return assignments                                                                                              
                                                                

In [375]:
assert len(random_centerpoints(inputs, 3)) == 3
assert compare_membership([1,2], [1,2])
assert not compare_membership([1,4], [1, 5])

In [376]:
from linear_algebra import vector_mean
from collections import defaultdict
k = 3
epochs = 5000 

def _k_means(k: int, epochs: int, inputs: List[Vector]) -> List[Vector]:
    """compute the k_means centerpoints for a set k based on a number of epochs"""
    centerpoints = random_centerpoints(inputs, k) # initialize 3 centerpoints randomnly
    membership = random_assignments(inputs, k) # initialize k random memberships
    for epoch in range(epochs): 
        assignments = defaultdict(list) 
        new_membership = [] 
        for input in inputs:
            min_distance = float("inf")
            for idx, cp in enumerate(centerpoints):      # over one input loop over all centerpoints
                distance = squared_distance(input, cp)   # compute dictance
                if distance < min_distance:              # compare distance to min distance
                    min_distance, index = distance, idx   
            new_membership.append(index)                        # for the smallest distance append the index for membership
            assignments[index].append(input)                    # for the smallest distance append index key and input vector
        assignments = assigments_check(assignments, k, inputs)  # check if allk keys are in assigment, if not assign key
        new_centerpoints = [vector_mean(points) for points in assignments.values()]  # compute vector_means for memberships
        
        if not compare_membership(new_membership, membership):  
            membership = new_membership
        else:
            #print(f" stopped at epoch {epoch}")
            break
        centerpoints = new_centerpoints   
    return sorted(centerpoints)

In [383]:
from typing import NamedTuple
from statistics import quantile


class KMeansTrial(NamedTuple):
    trial: int
    points: List[KMeansResult]

def K_means(k: int, inputs: List[Vector], n: int = 50, epochs: int = 100, ) -> List[Vector]:
    """Repeat _k_means n trials and output the .25 quantile trial k_means centerpoints for a set k 
    (attempts to )Prevents local minimum results being outputed"""
    
    trials = []
    for trial in range(n):
        trials.append(KMeansTrial(trial, _k_means(k, epochs, inputs)))
    
    trials = sorted(trials, key= lambda trial: trial.points)  # sort based on points
    return quantile(trials, p= 0.15).points                   # return the .25 quantile trial.points as the overal centerpoints

In [381]:
print(K_means(3, inputs))

[[-43.800000000000004, 5.4], [-15.888888888888888, -10.333333333333332], [18.333333333333332, 19.833333333333332]]


In [382]:
correct_centerpoints = [[-43.800000000000004, 5.4],
                        [-15.888888888888888, -10.333333333333332],
                        [18.333333333333332, 19.833333333333332]]

Still gets stuck in local minimun or maximum but overall good try :) 